In [14]:
import pandas as pd
import numpy as np
import os
from collections import Counter

In [8]:
df = pd.read_pickle('../output/06062022_clean_data.pkl')

In [9]:
df.head()

,name,comment,published_at,likes,Reply Count,parent_id,is_parent,comment_lower,clean_comments,impurity,year,month,lemmas,adjs_verbs,nouns,noun_phrases,adj_noun_phrases,Verb_noun_phrases,lemmasNgrams
0,schmoyoho,Stream the track! https://open.spotify.com/alb...,2018-07-12T18:42:05Z,13950,188,UgywoWu8TaqkOzGi8HF4AaABAg,Yes,stream the track! https://open.spotify.com/alb...,stream the track! (proceeds to dodsons gregor...,0.0,2018,07,"[stream, track, proceed, dodson, gregorys, ant...","[stream, great]","[track, proceed, dodson, antoine, cameo, gift,...",[bed_intruder],[great_gift],[],"[stream, track, proceed_dodson, gregorys, anto..."
1,Roweman,I wonder if they caught that intruder,2022-04-23T00:46:20Z,1,0,UgyUDUe6xUHoX5ziaql4AaABAg,Yes,i wonder if they caught that intruder,i wonder if they caught that intruder,0.0,2022,04,"[wonder, catch, intruder]","[wonder, catch]",[intruder],[],[],[],"[wonder_catch, intruder]"
2,xxCordellMxx,Why did I say this out loud when I was younger?,2022-04-21T20:27:23Z,0,0,Ugx9p0We_6_2c5etHQZ4AaABAg,Yes,why did i say this out loud when i was younger?,why did i say this out loud when i was younger?,0.0,2022,04,"[loud, young]",[young],[],[],[],[],"[loud, young]"
3,Nicholas DiGaetano,The rapist was never caught?,2022-04-21T04:17:40Z,0,0,Ugz-Y1LK6QpBOVv2WI54AaABAg,Yes,the rapist was never caught?,the rapist was never caught?,0.0,2022,04,"[rapist, catch]",[catch],[rapist],[],[],[],"[rapist, catch]"
4,Tango Bodhi,This will never get old.,2022-04-20T18:27:49Z,0,0,UgyHtE_H7Y5OOda5gfZ4AaABAg,Yes,this will never get old.,this will never get old.,0.0,2022,04,[old],[old],[],[],[],[],[old]


In [12]:
def compute_idf(df, column, preprocess=None, min_df=2):
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(set(tokens))

    # count tokens
    counter = Counter()
    df[column].map(update)
    # create DataFrame and compute idf
    idf_df = pd.DataFrame.from_dict(counter, orient='index', columns=['df'])
    idf_df = idf_df.query('df >= @min_df')
    idf_df['idf'] = np.log(len(df) / idf_df['df']) + 0.1
    idf_df.index.name = 'token'
    return idf_df

In [15]:
df_all_years = []
df['month'] = df['published_at'].apply(lambda x : x[5:7])
for year in df.year.unique():
    for month in df.month.unique():
        df_year = df[(df['year']==year) & (df['month']==month)]
        freq_df = compute_idf(df_year, column= 'lemmasNgrams')
        freq_df['year'] = year
        freq_df['month']= month
        freq_df['comment_count']= len(df_year)

        freq_df = freq_df.reset_index(drop=False)

        df_all_years.append(freq_df)


/var/folders/8r/h6d_m6c551n1g87q75lclwz40000gn/T/ipykernel_3776/1424019154.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idf_df['idf'] = np.log(len(df) / idf_df['df']) + 0.1


In [16]:
df_all_years = pd.concat(df_all_years)

In [17]:
df_all_years.head()

,token,df,idf,year,month,comment_count
0,antoine,3,4.791348,2018,07,327
1,bed_intruder,3,4.791348,2018,07,327
2,great,6,4.098201,2018,07,327
3,mom,2,5.196813,2018,07,327
4,actually,4,4.503666,2018,07,327


In [18]:
df_all_years.to_csv(os.path.join('../output/', '06072022_ngrams_by_years.csv')) # Output for Tableau Reports